In [ ]:
###########################
# Импорты и инициализации #
###########################

import cv2
import os
import glob
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing import image
from keras.optimizers import SGD

from keras.datasets import cifar10

# Задаем seed для повторяемости результатов
np.random.seed(42)
WIDTH = 60
HEIGHT = 77

In [ ]:
###################################################
# Считываем данные и подготавливаем их к обучению #
###################################################

list_of_files = []
target = []
i = -1
mapa = {}
for (dirpath, dirnames, filenames) in os.walk('Experiment/train_data'):
    for filename in filenames:
        if filename.endswith('.bmp'):
            target.append(i)
            list_of_files.append(os.sep.join([dirpath, filename]))
    mapa[i] = dirpath
    i += 1
print(mapa)
data = []
flag = True
for file_name in list_of_files:
    img = image.load_img(file_name, target_size = (WIDTH, HEIGHT), grayscale = True)
    tmp = image.img_to_array(img)
    tmp /= 255
    data.append(tmp)

train_data = np.array(data)
train_target = np_utils.to_categorical(target, 4)

In [ ]:
##############################
# Подготовка тестовых данных #
##############################

list_of_files = []
target = []
i = -1
mapa = {}
for (dirpath, dirnames, filenames) in os.walk('Experiment/test_data/test_data'):
    for filename in filenames:
        if filename.endswith('.bmp'):
            target.append(i)
            list_of_files.append(os.sep.join([dirpath, filename]))
    mapa[i] = dirpath
    i += 1
print(mapa)

data = []
for file_name in list_of_files:
    img = image.load_img(file_name, target_size = (WIDTH, HEIGHT), grayscale = True)
    tmp = image.img_to_array(img)
    tmp /= 255
    data.append(tmp)

test_data = np.array(data)
test_target = np_utils.to_categorical(target, 4)

In [ ]:
############################
# Создаем и обучаем модель #
############################

i = 0
DENSE = 500
DROPOUT = 30
ACTIVATION = 'relu'
WINDOW = 5
# for WINDOW in range(3, 8, 2):
#     for DROPOUT in range(20, 40, 5):
#         for DENSE in range(300, 700, 100):


# nb_filters = 84

# model = Sequential()

# model.add(Convolution2D(nb_filters, 7, 7,
#                  padding='valid',
#                  input_shape = (1, WIDTH, HEIGHT))) 
# model.add(Activation('tanh'))

# model.add(MaxPooling2D(pool_size = (2, 2), dim_ordering="tf"))

# model.add(Convolution2D(nb_filters, 4, 4))
# model.add(Activation('tanh'))
# model.add(MaxPooling2D(pool_size = (2, 2), dim_ordering="tf"))

# model.add(Convolution2D(nb_filters, 4, 4))
# model.add(Activation('tanh'))
# model.add(MaxPooling2D(pool_size = (2, 2), dim_ordering="tf"))

# left_branch = Sequential()
# left_branch.add(Convolution2D(54, 4, 4, input_shape = (1, 9, 9)))
# left_branch.add(Activation('tanh'))
# left_branch.add(MaxPooling2D(pool_size = (2, 2), dim_ordering="tf"))

# middle_branch = Sequential()
# middle_branch.add(Convolution2D(22, 4, 4, input_shape = (1, 9, 9)))
# middle_branch.add(Activation('tanh'))
# middle_branch.add(MaxPooling2D(pool_size = (3, 3), dim_ordering="tf"))

# right_branch = Sequential()
# right_branch.add(Convolution2D(8, 6, 6, input_shape = (1, 9, 9)))
# right_branch.add(Activation('tanh'))
# right_branch.add(MaxPooling2D(pool_size = (2, 2), dim_ordering="tf"))

# model.add(Flatten())
# model.add(Dense(300))
# model.add(Activation('tanh'))
# model.add(Dense(4))
# model.add(Activation('tanh'))
# sgd = SGD(lr = 0.001, decay = 0, momentum = 0, nesterov = True)

# model.compile(loss = 'categorical_crossentropy',
#               optimizer = sgd,
#               metrics = ['accuracy'])


# Создаем модель
model = Sequential()
model.add(Convolution2D(32, WINDOW, WINDOW, padding = 'same', input_shape = (1, WIDTH, HEIGHT), activation = 'relu'))
model.add(Convolution2D(32, WINDOW, WINDOW, activation = ACTIVATION))
model.add(MaxPooling2D(pool_size = (2, 2), dim_ordering="tf"))
model.add(Dropout(DROPOUT / 100))
model.add(Convolution2D(64, WINDOW, WINDOW, padding = 'same', activation = 'relu'))
model.add(Convolution2D(64, WINDOW, WINDOW, activation = ACTIVATION))
model.add(MaxPooling2D(pool_size = (2, 2), dim_ordering="tf"))
model.add(Dropout(DROPOUT / 100))
model.add(Flatten())
model.add(Dense(DENSE, activation = ACTIVATION))
model.add(Dropout(DROPOUT / 100))
model.add(Dense(4, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(train_data, train_target, batch_size = 32, epochs = 2, validation_split = 0.1, shuffle = True)

# Проверяем точность модели на тестовых данных
scores = model.evaluate(test_data, test_target, verbose = 0)
# Печатаем точность
i += 1
s = str(i) + ". Window: " + str(WINDOW) + ", Dropout: " + str(DROPOUT / 100) + ", Dense: ", str(DENSE), ", Activation: ", ACTIVATION
s2 = "Точность работы на тестовых данных: %.2f%%" %(scores[1] * 100)
with open("results.txt", "a") as myfile:
    myfile.write(str(s) + '\n' + str(s2) + '\n\n')
    
                
# print(i, ". Window: ", WINDOW, ", Dropout: %.2f" %(DROPOUT / 100), ", Dense: ", DENSE)
# print("Точность работы на тестовых данных: %.2f%%" %(scores[1] * 100))

In [ ]:
scores = model.evaluate(test_data, test_target, verbose = 0)
# Печатаем точность
i += 1
s = str(i) + ". Window: " + str(WINDOW) + ", Dropout: " + str(DROPOUT / 100) + ", Dense: ", str(DENSE), ", Activation: ", ACTIVATION
s2 = "Точность работы на тестовых данных: %.2f%%" %(scores[1] * 100)
with open("results.txt", "a") as myfile:
    myfile.write(str(s) + '\n' + str(s2) + '\n\n')